In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import keras
from keras.wrappers.scikit_learn import  KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense,Activation, Embedding, Flatten, LeakyReLU, BatchNormalization
from keras.layers import LeakyReLU,PReLU,ELU
from keras.activations import relu , sigmoid
from keras.layers import Dropout

In [9]:
data = pd.read_csv('datasets_13214_17927_Churn_Modelling.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [10]:
X = data.drop(['RowNumber','CustomerId','Surname','Exited'],axis = 1)
y = data['Exited']

In [11]:
geography = pd.get_dummies(X['Geography'],drop_first = True)
gender = pd.get_dummies(X['Gender'],drop_first = True)

In [12]:
X = pd.concat([X,geography,gender],axis=1)

In [13]:
X = X.drop(['Geography','Gender'],axis = 1)

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2,random_state = 0)

In [15]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [32]:
def create_model(layers,activation):
    model = Sequential()
    for i , nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes,input_dim = X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))

    model.add(Dense(units = 1,kernel_initializer='glorot_uniform', activation = 'sigmoid'))

    model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model

In [33]:
model = KerasClassifier(build_fn = create_model, verbose = 0)

In [34]:
layers = [(20), ( 40,20), (45,30,15)]
activations = ['sigmoid','relu']
param_grid = dict(layers = layers,activation = activations, batch_size = [128,256], epochs= [30])
grid = GridSearchCV(estimator = model, param_grid= param_grid,cv = 5)

grid_result = grid.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


In [37]:
print(grid_result.best_score_, grid_result.best_params_)

0.8572499990463257 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (45, 30, 15)}


In [38]:
pred_y = grid.predict(X_test)
y_pred = (pred_y > 0.5)


In [39]:
cm = confusion_matrix(y_pred, y_test)
score = accuracy_score(y_pred, y_test)

In [41]:
cm

array([[1543,  221],
       [  52,  184]])

In [42]:
score

0.8635